# Alchemix Code Review
## Intro
This document will contain the main information on the unpicking of the alchemix code. Additional/ supporting documents can be put in the 'alchemixStudy' folder.
This will serve as a central reference of our understanding of the alchemix smart contracts.
## Key questions
While understanding the code, we should pay special attention to the following, as they will be useful in our project:
* How are deposits and withdrawals handled?
* How are Yearn yields tracked? - this will be key for us as we must monitor yields coming in from yearn, to be able to show users their renumeration for performing rebalancing actions.
* Security - which security features have been implemented? Which seem different to standard (perhaps whitelisting)? Would these be useful for us?
* Which contracts can we leverage? - contracts such as wEth gateway and yearn token adaptor should be useful to us and we can implement them directly in a diamond pattern. Are there any others?
### Yearn Yields
function _preemptivelyHarvest(line 1002 AlchemistV2.sol)  
![preemptively harvest definition](./IMG/_preemptiveHarvest.png "preemptively harvest")  
Worth checking how yearn yield works - do number of yvault tokens increase over time?

### YieldTokenParams
A struct defined in IAlchemistV2State.sol defining yield token parameters.
It is inherited by IAlchemistV2 which is inherited by AlchemistV2.sol.
It kind of acts as a bit of a glossary of terms:  
![yield token parameters](./IMG/yieldTokenParams.png "yield token parameters")

## Deposit function flow
There are two options for depositing to alchemix:
* depositUnderlying -'Deposit an underlying token into the account of `recipient` as `yieldToken`'.
* deposit - 'Deposit a yield token into a user's account.'
These functions are broken down in the next few sections.
### depositUnderlying
deposit underlying is a function on both the WETHGateway contract and AlchemistV2 contract. 
#### WETHGateway implementation 
When a user uses the website to deposit, a transaction calling 'depositUnderlying' on the WETHGateway contract is created. This has a modifier _onlyWhitelisted which is described below.  
WETHGateway then call deposit on the WETH instance which is initialised in the constructor.  
Finally, depositUnderlying is called on the AlchemistV2 contract. 
##### _onlyWhitelisted()
OnlyWhitelisted is a function modifier which appears throughout the contracts.  
It first checks wether the message sender is an EOA (Externally Owned Account), controlled by public+private keys - basically a user wallet). If the caller is not an EOA, it must be a contract, and if it is not a whitelisted contract the transaction is reverted.
#### AlchemistV2 implementation
Checks:
* _onlyWhitelisted - see above
* _checkArgument(recipient != address(0)) - Checks the recipient is not the nulll address, reverting if it is
* _checkSupportedYieldToken(yieldToken) - checks the yield token is in the _supportedYieldTokens mapping.  

Calls:
* function calls _wrap (see below)
* function calls _deposit, depositing yield tokens to recipients account. The front end sets the recipient to the caller address.

_deposit is quite a beefy function, will break it down below.

##### _wrap
* loads yield token paramteters from _yieldTokens mapping
* creates an instance of the adapter using the ITokenAdapter interface
* safe transfers underlying tokens to AlchemistV2 contract.
* wraps tokens to 'wrapped shares' using adapter
* ensures wrapped shares amount > min amount out

## _deposit


### Shares
The alchemist contract keeps a record of deposits as shares instead of specific units of various tokens.  
The number of shares for a depsoit is calculated using '_convertYieldTokensToShares'. It takes a yieldToken address and uint256 amount as arguments. This is called by the function '_issueSharesForAmount'. If there are originally no shares, the number of yield tokens is the number of shares.  
Otherwise, shares is calculated as: return amount * _yieldTokens[yieldToken].totalShares / _calculateUnrealizedActiveBalance(yieldToken); (line 1611)


## Resources/ References
[Alchemix contracts outline](https://alchemix-finance.gitbook.io/v2/)  
[Alchemix V2 Contracts Github](https://github.com/alchemix-finance/v2-contracts)  
[Alchemix user docs](https://alchemix-finance.gitbook.io/user-docs/)  
[OpenAI prompt design docs](https://beta.openai.com/docs/guides/completion/prompt-design)  
